# Import libraries

In [ ]:
import os
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from scipy.ndimage import gaussian_filter
from collections import defaultdict



In [ ]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

In [ ]:
INPUT_FOLDER = "input_imgs"
OUTPUT_FOLDER = "lowpoly"

os.makedirs(INPUT_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
im_arr = None
points = None


In [ ]:
def read_process_input(filename):
	global im_arr
	im = Image.open(os.path.join(INPUT_FOLDER, filename))
	im_arr = np.array(im)
	# plt.imshow(im)
	perceptual_weight = np.array([0.2126, 0.7152, 0.0722])
	grayscale = (im * perceptual_weight).sum(axis=-1)
	plt.figure()
	plt.imshow(grayscale);

	x = gaussian_filter(grayscale, 2, mode="reflect")
	x2 = gaussian_filter(grayscale, 30, mode="reflect")

	# Take the difference, deweight negatives, normalise
	diff = (x - x2)
	diff[diff < 0] *= 0.1
	diff = np.sqrt(np.abs(diff) / diff.max())

	# plt.imshow(diff)
	return diff, im_arr

	

In [ ]:
def sample_im(ref, n=1000000):
	np.random.seed(0)
	h,w = ref.shape
	xs = np.random.randint(0, w, size=n)
	ys = np.random.randint(0, h, size=n)
	value = ref[ys,xs]
	accept = np.random.random(size=n) < value
	points = np.array([xs[accept], ys[accept]])
	return points.T, value[accept]

In [ ]:
def get_samples(diff):
	samples, v = sample_im(diff)
	plt.figure()
	plt.scatter(samples[:,0], -samples[:,1], c=v, s=0.2, edgecolors="none", cmap="viridis")

	return samples

In [ ]:
def get_colour_of_tri(tri, img):
	colours = defaultdict(lambda: [])
	h, w, _ = img.shape
	idcs = np.indices((w,h)).transpose(1,2,0).reshape(-1,2)
	index = tri.find_simplex(idcs).reshape(w,h)
	for i in range(0, w):
		for j in range(0,h):
			# index = tri.find_simplex((i,j))
			colours[index[i,j]].append(img[j,i,:])
	
	for index, array in colours.items():
		colours[int(index)] = np.array(array).mean(axis=0)
	return colours

In [ ]:
def load_plot_inp(filename):
	print("Normalized and highlighted:")
	diff, im_arr = read_process_input(filename)

	print("Scatter plot")
	samples = get_samples(diff)

	h,w,_ = im_arr.shape

	corners = np.array([(0,0), (0, h-1), (w-1, 0), (w-1, h-1)])
	points = np.concatenate((corners, samples))

	return points, im_arr, filename
	
	
	

In [ ]:
def generate_art(N, im_arr, points):
	n = 5 + N + 2 * int(N**2)
	print(n)

	tri = Delaunay(points[:n, :])
	colours = get_colour_of_tri(tri, im_arr)

	h,w, _ = im_arr.shape
	im_out = Image.new("RGB", (w,h))
	# print(im_out.size)
	im_draw = ImageDraw.Draw(im_out)
	for key,c in colours.items():
		t = tri.points[tri.simplices[key]]
		# t[:, 0], t[:, 1] = t[:, 1], t[:, 0].copy()
		# print(t, len(t))
		color = tuple(c.astype(np.uint8))
		im_draw.polygon(t.flatten().tolist(),fill=color, outline=color)

	# im_out = im_out.resize(im.size)
	plt.imshow(im_out)
	# im_out.save(f'lowpoly/out_{n}.png')
	return(im_out)

In [ ]:
w = interactive(load_plot_inp, filename=os.listdir(INPUT_FOLDER))

In [ ]:
display(w)

In [ ]:
points, im_arr, inp_name = w.result

In [ ]:
w2 = interactive(generate_art, N=widgets.IntSlider(min=1, max=100, step=1, value=45), im_arr=fixed(im_arr), points=fixed(points))

In [ ]:
display(w2)

In [ ]:
out = w2.result

# out.show()

In [ ]:
out

In [ ]:
out.save(f"{OUTPUT_FOLDER}/{inp_name}")